In [2]:
# Standard library imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

*Getting the data ready to be used with machine learning* <br />
<br />
Things that need to be done:
* splitting the data into features and labels (usually `X` and `y`)
* filling (imputing) or disregarding missing values
* converting non-numerical values to numerical values (feature encoding)

In [5]:
# Classification data
heart_disease = pd.read_csv("heart-disease.csv")
heart_disease.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


*Features*

In [7]:
X = heart_disease.drop("target", axis=1) # columns axis
X.head() # every single column without the target one

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2


*Labels*

In [9]:
y = heart_disease["target"]
y.head()

0    1
1    1
2    1
3    1
4    1
Name: target, dtype: int64

*Splitting the data into training and test sets*, as one of the fundemental Machine Learning principles dictates, that *models cannot be evaluated or tested with using the data they have learned from!*
<br /> Usually total samples are divided between 80% as a training data and 20% as testing data.

In [14]:
from sklearn.model_selection import train_test_split 

# Imported module returns 4 different values listed below
# Deciding that the test data set is 20% of the overall data 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

13 different features, 303 total samples, 80% of those (242) are the training set for the ML model, other 20% (61) are test set

In [17]:
X.shape, X_train.shape, X_test.shape, y_train.shape, y_test.shape

((303, 13), (242, 13), (61, 13), (242,), (61,))